In [1]:
import pandas as pd
import numpy as np 
from matplotlib import pyplot as plt

/usr/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [8]:
data = pd.read_csv("parfumo_data.csv")
data.dropna()

,brand,name,year,rating,votes,longevity,sillage,accord1,accord2
0,4160_Tuesdays,The Dark Heart Of Old Havana,2013.0,76.7568,37.0,79.5000,76.5000,sweet,spicy
1,4160_Tuesdays,Centrepiece,2015.0,81.5714,35.0,75.4167,67.8261,gourmand,sweet
2,4160_Tuesdays,Mother Nature s Naughty Daughters,2016.0,76.7742,31.0,83.9130,76.5217,fruity,floral
4,4160_Tuesdays,The Sexiest Scent on the Planet Ever IMHO,2013.0,72.9630,27.0,70.2174,63.0435,sweet,woody
5,4160_Tuesdays,Eau My Soul,2017.0,80.0000,20.0,79.4118,67.6471,spicy,woody
...,...,...,...,...,...,...,...,...,...
8456,Zoologist,squid,2019.0,80.8491,53.0,85.1020,79.7959,aquatic,spicy
8457,Zoologist,Dodo,2019.0,73.0769,52.0,86.2791,80.9302,green,spicy
8458,Zoologist,Panda 2017,2017.0,75.0000,51.0,77.6744,73.0233,green,fruity
8459,Zoologist,Chameleon,2019.0,75.8163,49.0,81.7073,74.3902,floral,sweet
